In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import classification_report as cr
from sklearn.model_selection import RandomizedSearchCV
from imblearn.combine import SMOTETomek
from collections import Counter

In [2]:
df=pd.read_csv('new_data\Stroke_analysis1.csv')
df

,Unnamed: 0,pid,age,gender,nhiss,mrs,systolic,distolic,glucose,paralysis,smoking,bmi,cholestrol,tos,risk
0,1,PID2829938,61,Male,0,-1,124,80,77,0,0,22,200,-1,0
1,2,PID1833441,75,Male,0,-1,123,82,79,0,0,20,208,-1,0
2,3,PID8745027,73,Male,0,-1,121,83,83,0,0,20,208,-1,0
3,4,PID4568062,63,Female,0,-1,120,85,105,0,0,20,210,-1,0
4,5,PID3855176,23,Male,0,-1,126,81,89,0,0,22,201,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4793,4794,PID11726,67,Male,13,6,180,83,198,1,2,39,219,3,2
4794,4795,PID11727,67,Female,20,3,180,88,188,0,2,22,187,1,2
4795,4796,PID11728,65,Male,6,4,126,111,227,0,3,23,233,3,2
4796,4797,PID11729,64,Male,19,5,126,82,262,2,2,28,244,1,2


In [3]:
df.drop(df.columns[:2],axis=1,inplace=True)
df

,age,gender,nhiss,mrs,systolic,distolic,glucose,paralysis,smoking,bmi,cholestrol,tos,risk
0,61,Male,0,-1,124,80,77,0,0,22,200,-1,0
1,75,Male,0,-1,123,82,79,0,0,20,208,-1,0
2,73,Male,0,-1,121,83,83,0,0,20,208,-1,0
3,63,Female,0,-1,120,85,105,0,0,20,210,-1,0
4,23,Male,0,-1,126,81,89,0,0,22,201,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4793,67,Male,13,6,180,83,198,1,2,39,219,3,2
4794,67,Female,20,3,180,88,188,0,2,22,187,1,2
4795,65,Male,6,4,126,111,227,0,3,23,233,3,2
4796,64,Male,19,5,126,82,262,2,2,28,244,1,2


In [4]:
df['gender'].replace(['Male','Female'],[0,1],inplace=True)
df

,age,gender,nhiss,mrs,systolic,distolic,glucose,paralysis,smoking,bmi,cholestrol,tos,risk
0,61,0,0,-1,124,80,77,0,0,22,200,-1,0
1,75,0,0,-1,123,82,79,0,0,20,208,-1,0
2,73,0,0,-1,121,83,83,0,0,20,208,-1,0
3,63,1,0,-1,120,85,105,0,0,20,210,-1,0
4,23,0,0,-1,126,81,89,0,0,22,201,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4793,67,0,13,6,180,83,198,1,2,39,219,3,2
4794,67,1,20,3,180,88,188,0,2,22,187,1,2
4795,65,0,6,4,126,111,227,0,3,23,233,3,2
4796,64,0,19,5,126,82,262,2,2,28,244,1,2


In [5]:
std_enc=StandardScaler()
X=df.drop('risk',axis=1).to_numpy()
y=df['risk'].to_numpy()

X=std_enc.fit_transform(X)
X

array([[ 0.5861116 , -0.7326905 , -1.60777799, ..., -1.88403202,
        -0.86502024, -3.07134719],
       [ 1.17712397, -0.7326905 , -1.60777799, ..., -2.20500584,
        -0.47028289, -3.07134719],
       [ 1.09269363, -0.7326905 , -1.60777799, ..., -2.20500584,
        -0.47028289, -3.07134719],
       ...,
       [ 0.75497228, -0.7326905 , -1.0755312 , ..., -1.72354511,
         0.7632713 ,  1.03920339],
       [ 0.71275711, -0.7326905 ,  0.07767017, ..., -0.92111057,
         1.30603514, -1.0160719 ],
       [ 0.75497228, -0.7326905 , -1.164239  , ..., -1.08159748,
         0.66458696,  1.03920339]])

In [6]:
df['risk'].value_counts()

2    3852
1     632
3     227
0      87
Name: risk, dtype: int64

In [20]:
X_train,X_temp,y_train,y_temp=tts(X,y,test_size=0.3,shuffle=True,stratify=y)
X_test,X_valid,y_test,y_valid=tts(X_temp,y_temp,test_size=1/3,shuffle=True,stratify=y_temp)
print(X_train.shape,X_valid.shape,X_test.shape)

(3358, 12) (480, 12) (960, 12)


In [21]:
smk = SMOTETomek(sampling_strategy='not majority', random_state=42)
X_train, y_train = smk.fit_resample(X_train, y_train)
# print(X_train)
print("After Oversampling:")
print("Risk", Counter(y_train))

After Oversampling:
Risk Counter({1: 2696, 2: 2696, 0: 2696, 3: 2696})


In [22]:
clf=RandomForestClassifier()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_valid)
print(cm(y_valid,y_pred))

[[  8   1   0   0]
 [  4  54   5   0]
 [  0   6 374   5]
 [  0   0   5  18]]


In [23]:
clf.score(X_test,y_test)

0.9427083333333334

In [24]:
print(cr(y_valid,y_pred))

              precision    recall  f1-score   support

           0       0.67      0.89      0.76         9
           1       0.89      0.86      0.87        63
           2       0.97      0.97      0.97       385
           3       0.78      0.78      0.78        23

    accuracy                           0.95       480
   macro avg       0.83      0.88      0.85       480
weighted avg       0.95      0.95      0.95       480



In [25]:
params={'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100,200,300,400,500,1000]}


search=RandomizedSearchCV(RandomForestClassifier(),params)
search.fit(np.concatenate((X_train,X_valid)),np.concatenate((y_train,y_valid)))
search.best_params_



{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_depth': 60,
 'bootstrap': False}

In [33]:
clf=RandomForestClassifier(**search.best_params_)
clf.fit(X_train,y_train)
clf.score(X_valid,y_valid)

0.94375

In [34]:
clf.score(X_test,y_test)

0.9447916666666667

In [35]:
print(cr(y_valid,clf.predict(X_valid)))

              precision    recall  f1-score   support

           0       0.73      0.89      0.80         9
           1       0.90      0.83      0.86        63
           2       0.97      0.97      0.97       385
           3       0.78      0.78      0.78        23

    accuracy                           0.94       480
   macro avg       0.84      0.87      0.85       480
weighted avg       0.94      0.94      0.94       480



In [36]:
print(cm(y_valid,clf.predict(X_valid)))

[[  8   1   0   0]
 [  3  52   8   0]
 [  0   5 375   5]
 [  0   0   5  18]]
